## Stock Sentiment Analysis using News Headlines

In [33]:
import pandas as pd

In [34]:
df=pd.read_csv('Data.csv', encoding = "ISO-8859-1")

In [35]:
stop_words = set({})
with open('stopwords','r') as f:
	for line in f:
		stop_words.add(line.strip())
df.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [36]:
index = df['Top23'].index
df['Top23'][index] = df['Top1'][index]

index = df['Top24'].index
df['Top24'][index] = df['Top1'][index]


index = df['Top25'].index
df['Top25'][index] = df['Top1'][index]

df.isnull().sum()

<ipython-input-36-75c4564ad5a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Top23'][index] = df['Top1'][index]
<ipython-input-36-75c4564ad5a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Top24'][index] = df['Top1'][index]
<ipython-input-36-75c4564ad5a4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Top25'][index] = df['Top1'][index]


Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    0
Top24    0
Top25    0
dtype: int64

In [40]:
df.duplicated().sum()

0

In [67]:
import re
re_obj = re.compile(r'[^a-zA-Z ]')
row_string = []


for i in range(len(df.index)):
	row_string.append(' '.join(top for top in df.iloc[i,2:]))
for i,_ in enumerate(row_string):
	# row_string[i].replace(r'[^a-zA-Z]','',regex = True,inplace=True)
	row_string[i] =  re_obj.sub('',row_string[i])
	row_string[i] = row_string[i].lower()
row_string[0]

'a hindrance operations extracts leaked reports scorecard hughes instant hit buoys blues jack gets skates icecold alex chaos maracana builds united depleted leicester prevail elliott spoils evertons party hungry spurs sense rich pickings gunners wide easy target derby raise glass strupars debut double southgate strikes leeds pay penalty hammers hand robson youthful lesson saints party like wear wolves turned lambs stump mike catches testy goughs taunt langer escapes hit flintoff injury piles woe england hunters threaten jospin new battle somme kohls successor drawn scandal the difference men women sara denver nurse turned solicitor dianas landmine crusade put tories panic yeltsins resignation caught opposition flatfooted a hindrance operations extracts leaked reports a hindrance operations extracts leaked reports a hindrance operations extracts leaked reports'

In [62]:
def remove_stop(sentence : str):
	words = sentence.split()
	return ' '.join([word  for word in words if word not in stop_words])


for i,s in enumerate(row_string):
	row_string[i] = remove_stop(row_string[i])

row_string[0]

# 	df.iloc[:,2 + i] = df.iloc[:,2 + i].map(remove_stop)


'A hindrance operations extracts leaked reports Scorecard Hughes instant hit buoys Blues Jack gets skates icecold Alex Chaos Maracana builds United Depleted Leicester prevail Elliott spoils Evertons party Hungry Spurs sense rich pickings Gunners wide easy target Derby raise glass Strupars debut double Southgate strikes Leeds pay penalty Hammers hand Robson youthful lesson Saints party like Wear wolves turned lambs Stump mike catches testy Goughs taunt Langer escapes hit Flintoff injury piles woe England Hunters threaten Jospin new battle Somme Kohls successor drawn scandal The difference men women Sara Denver nurse turned solicitor Dianas landmine crusade put Tories panic Yeltsins resignation caught opposition flatfooted A hindrance operations extracts leaked reports A hindrance operations extracts leaked reports A hindrance operations extracts leaked reports'

In [69]:
df['Top1'] = pd.Series(row_string)
df = df[['Date','Label','Top1']]
df.columns = ['Date','Label','Content']
df.head()

,Date,Label,Content
0,2000-01-03,0,a hindrance operations extracts leaked reports...
1,2000-01-04,0,scorecard the best lake scene leader german sl...
2,2000-01-05,0,coventry caught counter flo uniteds rivals roa...
3,2000-01-06,1,pilgrim knows progress thatcher facing ban mci...
4,2000-01-07,1,hitches horlocks beckham united survive breast...


In [70]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [73]:
headlines = list(train['Content'])

'scorecard the best lake scene leader german sleaze inquiry cheerio boyo the main recommendations has cubie killed fees has cubie killed fees has cubie killed fees hopkins furious fosters lack hannibal appetite has cubie killed fees a tale two tails i say i like i like i say elbows eyes nipples task force assess risk asteroid collision how i found last on critical list the timing lives dear doctor irish court halts ira mans extradition northern ireland burundi peace initiative fades rebels reject mandela mediator pe points way forward ecb campaigners keep pressure nazi war crimes suspect scorecard scorecard scorecard'

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [36]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [37]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [38]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

In [39]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [40]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[138  48]
 [  9 183]]
0.8492063492063492
              precision    recall  f1-score   support

           0       0.94      0.74      0.83       186
           1       0.79      0.95      0.87       192

    accuracy                           0.85       378
   macro avg       0.87      0.85      0.85       378
weighted avg       0.86      0.85      0.85       378



In [41]:
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression

basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(traindataset, train["Label"])


In [42]:
preds1 = basicmodel.predict(test_dataset)

acc1=accuracy_score(test['Label'], preds1)
print(f'logic regression [{acc1}]')

logic regression [0.8571428571428571]


In [43]:
# * naive beyes 


from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(headlines)



In [47]:
from sklearn.naive_bayes import MultinomialNB

advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds4)
print(f'naive beyes :  [{acc2}]')

naive beyes :  [0.5079365079365079]


In [49]:

# gradient boosting machine

advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.9, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(headlines)


In [51]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds8 = advancedmodel.predict(advancedtest.toarray())
acc3 = accuracy_score(test['Label'], preds8)

print(f'naive beyes :  [{acc3}]')

naive beyes :  [0.7116402116402116]


In [60]:
advancedvectorizer = TfidfVectorizer( min_df=0.2, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(headlines)

In [61]:
# stochastic gradient descent   


from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression

advancedmodel = SGDClassifier(loss='modified_huber',random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds10 = advancedmodel.predict(advancedtest.toarray())
acc4 = accuracy_score(test['Label'], preds10)


print(f'na :  [{acc4}]')

naive beye :  [0.5079365079365079]


In [ ]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2


tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(headlines)
sequences_train = tokenizer.texts_to_sequences(headlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

In [2]:
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU


X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=3,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds5 = model.predict_classes(X_test, verbose=0)
acc15 = accuracy_score(test['Label'], preds5)


print(f'LSTM : {preds5}')

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`